# Imports

In [49]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import tensorflow as tf
import shap
from sklearn.svm import SVR
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing

os.environ["KERAS_BACKEND"] = "jax"

from keras import Sequential
from keras.src.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from pmdarima.arima import auto_arima
from pyESN import ESN
from shap.plots import colors
from sklearn.ensemble import RandomForestRegressor as RandomForest
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from statsmodels.tsa.api import SimpleExpSmoothing
from xgboost import XGBRegressor

SEED = 100


def reset_seed(rnd_seed=SEED):
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(rnd_seed)
    np.random.seed(rnd_seed)
    tf.random.set_seed(rnd_seed)


reset_seed()


def rrmse(actual, predicted) -> int:
    return root_mean_squared_error(actual, predicted) / np.mean(actual)


def smape(actual, predicted) -> int:
    if not all([isinstance(actual, np.ndarray),
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)

    return round(
        np.mean(
            np.abs(predicted - actual) /
            ((np.abs(predicted) + np.abs(actual)) / 2)
        ) * 100, 2
    )



# Load Datasets



In [50]:
df = pd.read_csv('dataset - Palmas/elect_merged_selected.csv', sep=";", decimal=",", header=0)
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
df = df[df["date"] >= '2019-01-01']
df = df[df["date"] <= '2023-12-31']

df.set_index("date", inplace=True)

df = df.astype("float32")

x = df.drop("consumption", axis=1)
y = df["consumption"]

df

consumption     covid   LAG_01   LAG_02   LAG_03   LAG_06  \
date                                                                    
2019-01-31      10732.0  0.000000  20426.0  19998.0  20174.0  17751.0   
2019-02-28      17208.0  0.000000  10732.0  20426.0  19998.0  22575.0   
2019-03-31      19451.0  0.000000  17208.0  10732.0  20426.0  21969.0   
2019-04-30      23949.0  0.000000  19451.0  17208.0  10732.0  20174.0   
2019-05-31      24015.0  0.000000  23949.0  19451.0  17208.0  19998.0   
2019-06-30      25339.0  0.000000  24015.0  23949.0  19451.0  20426.0   
2019-07-31      17866.0  0.000000  25339.0  24015.0  23949.0  10732.0   
2019-08-31      20875.0  0.000000  17866.0  25339.0  24015.0  17208.0   
2019-09-30      24787.0  0.000000  20875.0  17866.0  25339.0  19451.0   
2019-10-31      20567.0  0.000000  24787.0  20875.0  17866.0  23949.0   
2019-11-30      23507.0  0.000000  20567.0  24787.0  20875.0  24015.0   
2019-12-31      23408.0  0.000000  23507.0  20567.0  24787.0  25339.0   
2020-01-31       8685.0  0.000000  23408.0  23507.0  20567.0  17866.0   
2020-02-29      17827.0  0.000000   8685.0  23408.0  23507.0  20875.0   
2020-03-31      20919.0  0.290323  17827.0   8685.0  23408.0  24787.0   
2020-04-30      12363.0  1.000000  20919.0  17827.0   8685.0  20567.0   
2020-05-31      12430.0  1.000000  12363.0  20919.0  17827.0  23507.0   
2020-06-30      12592.0  1.000000  12430.0  12363.0  20919.0  23408.0   
2020-07-31      12086.0  1.000000  12592.0  12430.0  12363.0   8685.0   
2020-08-31      12340.0  1.000000  12086.0  12592.0  12430.0  17827.0   
2020-09-30      11651.0  1.000000  12340.0  12086.0  12592.0  20919.0   
2020-10-31       7252.0  1.000000  11651.0  12340.0  12086.0  12363.0   
2020-11-30       8091.0  1.000000   7252.0  11651.0  12340.0  12430.0   
2020-12-31       9746.0  1.000000   8091.0   7252.0  11651.0  12592.0   
2021-01-31      11377.0  1.000000   9746.0   8091.0   7252.0  12086.0   
2021-02-28       9494.0  1.000000  11377.0   9746.0   8091.0  12340.0   
2021-03-31       9738.0  1.000000   9494.0  11377.0   9746.0  11651.0   
2021-04-30      10678.0  1.000000   9738.0   9494.0  11377.0   7252.0   
2021-05-31      13089.0  1.000000  10678.0   9738.0   9494.0   8091.0   
2021-06-30      14538.0  1.000000  13089.0  10678.0   9738.0   9746.0   
2021-07-31      14145.0  1.000000  14538.0  13089.0  10678.0  11377.0   
2021-08-31      13681.0  1.000000  14145.0  14538.0  13089.0   9494.0   
2021-09-30      12361.0  1.000000  13681.0  14145.0  14538.0   9738.0   
2021-10-31      11683.0  1.000000  12361.0  13681.0  14145.0  10678.0   
2021-11-30      19012.0  1.000000  11683.0  12361.0  13681.0  13089.0   
2021-12-31      18837.0  1.000000  19012.0  11683.0  12361.0  14538.0   
2022-01-31      12843.0  1.000000  18837.0  19012.0  11683.0  14145.0   
2022-02-28      13786.0  1.000000  12843.0  18837.0  19012.0  13681.0   
2022-03-31      22827.0  1.000000  13786.0  12843.0  18837.0  12361.0   
2022-04-30      15589.0  0.633333  22827.0  13786.0  12843.0  11683.0   
2022-05-31      15591.0  0.000000  15589.0  22827.0  13786.0  19012.0   
2022-06-30      15575.0  0.000000  15591.0  15589.0  22827.0  18837.0   
2022-07-31      13855.0  0.000000  15575.0  15591.0  15589.0  12843.0   
2022-08-31      14372.0  0.000000  13855.0  15575.0  15591.0  13786.0   
2022-09-30      12741.0  0.000000  14372.0  13855.0  15575.0  22827.0   
2022-10-31      13622.0  0.000000  12741.0  14372.0  13855.0  15589.0   
2022-11-30      14326.0  0.000000  13622.0  12741.0  14372.0  15591.0   
2022-12-31      12500.0  0.000000  14326.0  13622.0  12741.0  15575.0   
2023-01-31      10272.0  0.000000  12500.0  14326.0  13622.0  13855.0   
2023-02-28      13555.0  0.000000  10272.0  12500.0  14326.0  14372.0   
2023-03-31      15342.0  0.000000  13555.0  10272.0  12500.0  12741.0   
2023-04-30      14480.0  0.000000  15342.0  13555.0  10272.0  13622.0   
2023-05-31      16964.0  0.000000  14480.0  15342.0  13555.0  14326.0 

# Best Params

In [51]:
best = {}
for model in ["CNN", "ESN", "LSTM", "RF", "SVR", "XGB"]:
    df_aux = pd.DataFrame()
    for optimizer in ["GA", "PSO"]:
        for seed, name in [("1000", "A"), ("2000", "B"), ("3000", "C")]:
            try:
                new_df = pd.read_csv(f'params/{optimizer}-{model} ITERS SEED {seed}.csv', sep=";", decimal=",",
                                     header=0)
                df_aux = pd.concat([df_aux, new_df], axis=0)
            except Exception as e:
                continue

    df_aux = df_aux.sort_values(by=["Fitness"])
    df_aux[df_aux.isnull()] = None
    best[f"{model}"] = df_aux[:1].iloc[0].to_dict()

# Prediction
## Model Instantiation

In [52]:
#LSTM
tf.keras.backend.clear_session()
lstm = Sequential([
    Input((x.shape[1], 1)),
    LSTM(best["LSTM"]["Units"],
         activation=best["LSTM"]["Activation"],
         use_bias=best["LSTM"]["Bias"],
         seed=SEED),
    Dense(1),
])
lstm.compile(loss='mse')

#RF
rf = RandomForest(random_state=SEED,
                  n_estimators=int(best["RF"]["N_estimators"]),
                  max_depth=int(best["RF"]["Max_depth"]),
                  min_samples_split=int(best["RF"]["Min_samples_split"]),
                  min_samples_leaf=int(best["RF"]["Min_samples_leaf"]))

#SVR
svr = SVR(C=best["SVR"]["C"],
          epsilon=best["SVR"]["Epsilon"],
          kernel=best["SVR"]["Kernel"])

#XGB
updater = "coord_descent" if best["XGB"]["Booster"] == "gblinear" else None
xgb = XGBRegressor(random_state=SEED,
                   n_estimators=int(best["XGB"]["N_estimators"]),
                   max_depth=int(best["XGB"]["Max_depth"]),
                   booster=best["XGB"]["Booster"],
                   reg_lambda=best["XGB"]["Lambda"],
                   reg_alpha=best["XGB"]["Alpha"],
                   updater=updater)

#CNN
tf.keras.backend.clear_session()
cnn = Sequential([
    Input((x.shape[1], 1)),
    Conv1D(filters=58,
           kernel_size=2),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(442),
    Dense(1),
])
cnn.compile(loss='mse')

#ESN
esn = ESN(n_inputs=x.shape[1],
          n_outputs=1,
          n_reservoir=464,
          sparsity=0.1,
          spectral_radius=2.0,
          noise=0.0001,
          random_state=SEED)

## Train Split

In [53]:
reset_seed()
N_SPLITS = 5
H = 12
i = 1

preds_cnn = []
preds_esn = []
preds_lstm = []
preds_rf = []
preds_svr = []
preds_xgb = []

scores_cnn = []
scores_esn = []
scores_lstm = []
scores_rf = []
scores_svr = []
scores_xgb = []

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=H, shuffle=False)
for i_train, i_test in TimeSeriesSplit(n_splits=N_SPLITS, test_size=1).split(x_train, y_train):
    reset_seed()
    kx_train, kx_test = x_train.iloc[i_train].to_numpy(), x_train.iloc[i_test].to_numpy()
    ky_train, ky_test = y_train.iloc[i_train].to_numpy(), y_train.iloc[i_test].to_numpy()

    cnn.fit(kx_train, ky_train, shuffle=False, verbose=False, epochs=198)
    tf.keras.backend.clear_session()
    esn.fit(kx_train, ky_train)
    tf.keras.backend.clear_session()
    lstm.fit(kx_train, ky_train, shuffle=False, verbose=False, epochs=best["LSTM"]["Epochs"],
             batch_size=best["LSTM"]["Batch Size"])
    rf.fit(kx_train, ky_train)
    svr.fit(kx_train, ky_train)
    xgb.fit(kx_train, ky_train)

    pred_cnn = int(cnn.predict(kx_test)[0][0])
    pred_esn = int(esn.predict(kx_test)[0])
    pred_lstm = int(lstm.predict(kx_test)[0][0])
    pred_rf = int(rf.predict(kx_test)[0])
    pred_svr = int(svr.predict(kx_test)[0])
    pred_xgb = int(xgb.predict(kx_test)[0])

    preds_cnn.append(pred_cnn)
    preds_esn.append(pred_esn)
    preds_lstm.append(pred_lstm)
    preds_rf.append(pred_rf)
    preds_svr.append(pred_svr)
    preds_xgb.append(pred_xgb)

    scores_cnn.append(mean_absolute_error(ky_test, [pred_cnn]))
    scores_esn.append(mean_absolute_error(ky_test, [pred_esn]))
    scores_lstm.append(mean_absolute_error(ky_test, [pred_lstm]))
    scores_rf.append(mean_absolute_error(ky_test, [pred_rf]))
    scores_svr.append(mean_absolute_error(ky_test, [pred_svr]))
    scores_xgb.append(mean_absolute_error(ky_test, [pred_xgb]))

    i = i + 1

cnn.fit(x_train.to_numpy(), y_train.to_numpy(), shuffle=False, verbose=False, epochs=198)
lstm.fit(x_train.to_numpy(), y_train.to_numpy(), shuffle=False, verbose=False, epochs=best["LSTM"]["Epochs"],
         batch_size=best["LSTM"]["Batch Size"])
rf.fit(x_train.to_numpy(), y_train.to_numpy())
svr.fit(x_train.to_numpy(), y_train.to_numpy())
xgb.fit(x_train.to_numpy(), y_train.to_numpy())

df = pd.DataFrame({
    "CNN": pd.Series(scores_cnn),
    "ESN": pd.Series(scores_esn),
    "LSTM": pd.Series(scores_lstm),
    "RF": pd.Series(scores_rf),
    "SVR": pd.Series(scores_svr),
    "XGB": pd.Series(scores_xgb),
})
df.to_csv("results - Palmas 12m/scores.csv", sep=";", decimal=",", header=True)
display(df)

1/1 [==============================] - 0s 307ms/step


/tmp/ipykernel_423740/2059606370.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_esn = int(esn.predict(kx_test)[0])


1/1 [==============================] - 0s 100ms/step


/tmp/ipykernel_423740/2059606370.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_esn = int(esn.predict(kx_test)[0])


1/1 [==============================] - 0s 67ms/step


...

/tmp/ipykernel_423740/2059606370.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_esn = int(esn.predict(kx_test)[0])


1/1 [==============================] - 0s 76ms/step


/tmp/ipykernel_423740/2059606370.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_esn = int(esn.predict(kx_test)[0])


1/1 [==============================] - 0s 159ms/step


CNN     ESN    LSTM      RF    SVR     XGB
0  1024.0  3166.0  7482.0   316.0  614.0   472.0
1  3144.0   278.0  1729.0  2927.0  941.0  1519.0
2    29.0  5290.0  2691.0   358.0  347.0  3107.0
3  1274.0  1923.0  5892.0  1028.0  945.0  2027.0
4  1416.0  4400.0  1857.0   634.0  947.0  2271.0

## Test Split

In [54]:

preds_cnn = []
preds_esn = []
preds_lstm = []
preds_rf = []
preds_svr = []
preds_xgb = []
preds_single_brown = []
preds_double_brown = []
preds_hw_additive = []
preds_hw_mult = []

datasets_cnn = pd.concat([x_train.copy(), y_train.copy()], axis=1)
datasets_esn = pd.concat([x_train.copy(), y_train.copy()], axis=1)
datasets_lstm = pd.concat([x_train.copy(), y_train.copy()], axis=1)
datasets_rf = pd.concat([x_train.copy(), y_train.copy()], axis=1)
datasets_svr = pd.concat([x_train.copy(), y_train.copy()], axis=1)
datasets_xgb = pd.concat([x_train.copy(), y_train.copy()], axis=1)

for i_test in range(len(x_test)):
    sx_test = x_test.iloc[[i_test]].astype(float)
    sx_test_aux = sx_test.copy()

    #Get the lags for each model based in the last prediction
    datasets_cnn = pd.concat([datasets_cnn, sx_test_aux], axis=0)
    for lag in range(1, 12 + 1):
        if 'LAG_' + "{:02d}".format(lag) in sx_test.columns:
            sx_test_aux[f'LAG_' + '{:02d}'.format(lag)] = datasets_cnn["consumption"].shift(lag)
            datasets_cnn[f'LAG_' + '{:02d}'.format(lag)] = datasets_cnn["consumption"].shift(lag)

    pred = int(cnn.predict(sx_test_aux.to_numpy())[0][0])

    sx_test_aux["consumption"] = pred
    preds_cnn.append(pred)
    datasets_cnn.update(sx_test_aux)

for i_test in range(len(x_test)):
    sx_test = x_test.iloc[[i_test]].astype(float)
    sx_test_aux = sx_test.copy()

    #Get the lags for each model based in the last prediction
    datasets_esn = pd.concat([datasets_esn, sx_test_aux], axis=0)
    for lag in range(1, 12 + 1):
        if 'LAG_' + "{:02d}".format(lag) in sx_test.columns:
            sx_test_aux[f'LAG_' + '{:02d}'.format(lag)] = datasets_esn["consumption"].shift(lag)
            datasets_esn[f'LAG_' + '{:02d}'.format(lag)] = datasets_esn["consumption"].shift(lag)

    pred = int(esn.predict(sx_test_aux.to_numpy())[0])

    sx_test_aux["consumption"] = pred
    preds_esn.append(pred)
    datasets_esn.update(sx_test_aux)

for i_test in range(len(x_test)):
    sx_test = x_test.iloc[[i_test]].astype(float)
    sx_test_aux = sx_test.copy()

    #Get the lags for each model based in the last prediction
    datasets_lstm = pd.concat([datasets_lstm, sx_test_aux], axis=0)
    for lag in range(1, 12 + 1):
        if 'LAG_' + "{:02d}".format(lag) in sx_test.columns:
            sx_test_aux[f'LAG_' + '{:02d}'.format(lag)] = datasets_lstm["consumption"].shift(lag)
            datasets_lstm[f'LAG_' + '{:02d}'.format(lag)] = datasets_lstm["consumption"].shift(lag)

    pred = int(lstm.predict(sx_test_aux.to_numpy())[0][0])

    sx_test_aux["consumption"] = pred
    preds_lstm.append(pred)
    datasets_lstm.update(sx_test_aux)

for i_test in range(len(x_test)):
    sx_test = x_test.iloc[[i_test]].astype(float)
    sx_test_aux = sx_test.copy()

    #Get the lags for each model based in the last prediction
    datasets_rf = pd.concat([datasets_rf, sx_test_aux], axis=0)
    for lag in range(1, 12 + 1):
        if 'LAG_' + "{:02d}".format(lag) in sx_test.columns:
            sx_test_aux[f'LAG_' + '{:02d}'.format(lag)] = datasets_rf["consumption"].shift(lag)
            datasets_rf[f'LAG_' + '{:02d}'.format(lag)] = datasets_rf["consumption"].shift(lag)

    pred = int(rf.predict(sx_test_aux.to_numpy())[0])

    sx_test_aux["consumption"] = pred
    preds_rf.append(pred)
    datasets_rf.update(sx_test_aux)

for i_test in range(len(x_test)):
    sx_test = x_test.iloc[[i_test]].astype(float)
    sx_test_aux = sx_test.copy()

    #Get the lags for each model based in the last prediction
    datasets_svr = pd.concat([datasets_svr, sx_test_aux], axis=0)
    for lag in range(1, 12 + 1):
        if 'LAG_' + "{:02d}".format(lag) in sx_test.columns:
            sx_test_aux[f'LAG_' + '{:02d}'.format(lag)] = datasets_svr["consumption"].shift(lag)
            datasets_svr[f'LAG_' + '{:02d}'.format(lag)] = datasets_svr["consumption"].shift(lag)

    pred = int(svr.predict(sx_test_aux.to_numpy())[0])

    sx_test_aux["consumption"] = pred
    preds_svr.append(pred)
    datasets_svr.update(sx_test_aux)

for i_test in range(len(x_test)):
    sx_test = x_test.iloc[[i_test]].astype(float)
    sx_test_aux = sx_test.copy()

    #Get the lags for each model based in the last prediction
    datasets_xgb = pd.concat([datasets_xgb, sx_test_aux], axis=0)
    for lag in range(1, 12 + 1):
        if 'LAG_' + "{:02d}".format(lag) in sx_test.columns:
            sx_test_aux[f'LAG_' + '{:02d}'.format(lag)] = datasets_xgb["consumption"].shift(lag)
            datasets_xgb[f'LAG_' + '{:02d}'.format(lag)] = datasets_xgb["consumption"].shift(lag)

    pred = int(xgb.predict(sx_test_aux.to_numpy())[0])

    sx_test_aux["consumption"] = pred
    preds_xgb.append(pred)
    datasets_xgb.update(sx_test_aux)

single_brown = SimpleExpSmoothing(y_train)
double_brown = Holt(y_train)
hw_additive = ExponentialSmoothing(y_train, seasonal_periods=12, trend='add', seasonal='add')
hw_mult = ExponentialSmoothing(y_train, seasonal_periods=12, trend='add', seasonal='mul')
arima = auto_arima(y_train, start_p=0, start_P=0, d=1, start_q=0, start_Q=0, m=12, max_p=5, max_P=5, max_q=5, max_Q=5,
                   fits=50, seasonal=False, random_state=SEED)
display(arima.summary())

preds_single_brown = single_brown.fit().forecast(H)
preds_double_brown = double_brown.fit().forecast(H)
preds_hw_additive = hw_additive.fit().forecast(H)
preds_hw_mult = hw_mult.fit().forecast(H)
preds_arima = arima.predict(n_periods=H)

plt.figure(figsize=(12, 4))
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams.update({'font.size': 12})
plt.rcParams['axes.prop_cycle'] = plt.cycler(
    color=["blue", "green", "darkgoldenrod", colors.red_rgb, "purple", "cyan", "slategrey", "coral"])

plt.plot(x_test.index, preds_lstm, label="GA-LSTM")
plt.plot(x_test.index, preds_rf, label="GA-RF")
plt.plot(x_test.index, preds_svr, label="GA-SVR")
plt.plot(x_test.index, preds_xgb, label="GA-XGB")
plt.plot(x_test.index, y_test, label="Expected", color="black")

plt.xlabel('Months')
plt.ylabel('Consumption (KWh)')

ax = plt.gca()
ax.set_facecolor('white')
plt.xticks(x_test.index, x_test.index.strftime("%m/%y"))
plt.grid(True, color='grey', linestyle="--", linewidth=0.5)
plt.legend(facecolor='white', )
plt.savefig("results - Palmas 12m/Forecast Results.png", bbox_inches='tight')
plt.show()


1/1 [==============================] - 0s 39ms/step


/tmp/ipykernel_423740/1390058064.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = int(esn.predict(sx_test_aux.to_numpy())[0])
/tmp/ipykernel_423740/1390058064.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = int(esn.predict(sx_test_aux.to_numpy())[0])
/tmp/ipykernel_423740/1390058064.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = int(esn.predict(sx_test_aux.to_numpy())[0])
/tmp/ipykernel_423740/1390058064.py:47: DeprecationWarning

1/1 [==============================] - 0s 58ms/step


/home/eduardo/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/home/eduardo/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/home/eduardo/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/home/eduardo/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/home/eduardo/miniconda3/envs/rapids-24.06/lib/python3.11/si

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   48
Model:               SARIMAX(0, 1, 0)   Log Likelihood                -458.981
Date:                Fri, 06 Sep 2024   AIC                            919.962
Time:                        20:08:02   BIC                            921.812
Sample:                    01-31-2019   HQIC                           920.658
                         - 12-31-2022                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2       1.74e+07   2.38e+06      7.309      0.000    1.27e+07    2.21e+07
===================================================================================
Ljung-Box (L1) (Q):                   3.52   Jarque-Bera (JB):                15.64
Prob(Q):                              0.06   Prob(JB):                         0.00
Heteroskedasticity (H):               0.42   Skew:                            -0.76
Prob(H) (two-sided):                  0.10   Kurtosis:                         5.39
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

/home/eduardo/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/eduardo/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/eduardo/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [55]:
df_measures = pd.DataFrame(index=["MAE", "sMAPE"], columns=["CNN", "ESN", "LSTM", "RF", "SVR", "XGB"])
df_measures["CNN"] = pd.Series(
    [mean_absolute_error(y_test, preds_cnn), smape(y_test, preds_cnn)],
    index=["MAE", "sMAPE"])
df_measures["ESN"] = pd.Series(
    [mean_absolute_error(y_test, preds_esn), smape(y_test, preds_esn)],
    index=["MAE", "sMAPE"])
df_measures["LSTM"] = pd.Series(
    [mean_absolute_error(y_test, preds_lstm), smape(y_test, preds_lstm)],
    index=["MAE", "sMAPE"])
df_measures["RF"] = pd.Series([mean_absolute_error(y_test, preds_rf), smape(y_test, preds_rf)],
                              index=["MAE", "sMAPE"])
df_measures["SVR"] = pd.Series(
    [mean_absolute_error(y_test, preds_svr), smape(y_test, preds_svr)], index=["MAE", "sMAPE"])
df_measures["XGB"] = pd.Series(
    [mean_absolute_error(y_test, preds_xgb), smape(y_test, preds_xgb)], index=["MAE", "sMAPE"])
df_measures["Single Exponential Smoothing "] = pd.Series(
    [mean_absolute_error(y_test, preds_single_brown), smape(y_test, preds_single_brown)], index=["MAE", "sMAPE"])
df_measures["Double Exponential Smoothing"] = pd.Series(
    [mean_absolute_error(y_test, preds_double_brown), smape(y_test, preds_double_brown)], index=["MAE", "sMAPE"])
df_measures["Triple Exponential Smoothing Additive"] = pd.Series(
    [mean_absolute_error(y_test, preds_hw_additive), smape(y_test, preds_hw_additive)], index=["MAE", "sMAPE"])
df_measures["Triple Exponential Smoothing Multiplicative"] = pd.Series(
    [mean_absolute_error(y_test, preds_hw_mult), smape(y_test, preds_hw_mult)], index=["MAE", "sMAPE"])
df_measures["ARIMA"] = pd.Series(
    [mean_absolute_error(y_test, preds_arima), smape(y_test, preds_arima)], index=["MAE", "sMAPE"])

df_measures.to_csv(f"results - Palmas 12m/Measures.csv", sep=";", decimal=",", index=True)
df_measures

CNN          ESN     LSTM           RF          SVR      XGB  \
MAE    3411.00  4198.666667  2264.50  3211.333333  3247.416667  3760.75   
sMAPE    23.33    27.430000    15.35    21.830000    22.120000    24.45   

       Single Exponential Smoothing   Double Exponential Smoothing  \
MAE                      3207.777988                   4190.560075   
sMAPE                      21.820000                     30.100000   

       Triple Exponential Smoothing Additive  \
MAE                              5415.740503   
sMAPE                              43.310000   

       Triple Exponential Smoothing Multiplicative        ARIMA  
MAE                                    4534.491345  3322.166667  
sMAPE                                    33.840000    22.650000

# SHAP Values - Random Forest

In [56]:
shap.initjs()

explainer_rf = shap.Explainer(rf)
shap_rf = explainer_rf(x_train)

base_values_rf = pd.DataFrame(index=x_train.columns)
values_rf = pd.DataFrame(index=x_train.columns)
data_rf = pd.DataFrame(index=x_train.columns)

for sample in shap_rf:
    base_values_rf = pd.concat([base_values_rf, pd.Series(sample.base_values, index=x_train.columns)], axis=1)
    values_rf = pd.concat([values_rf, pd.Series(sample.values, index=x_train.columns)], axis=1)
    data_rf = pd.concat([data_rf, pd.Series(sample.data, index=x_train.columns)], axis=1)

force = shap.plots.force(base_value=base_values_rf.mean(axis=1).to_numpy(),
                         shap_values=values_rf.mean(axis=1).to_numpy(),
                         text_rotation=90,
                         plot_cmap=["#ff0051", "#3838ff"],
                         features=data_rf.mean(axis=1).to_numpy().astype(int),
                         feature_names=x_train.columns,
                         show=True)
shap.save_html("results - Palmas 12m/SHAPE FORCE RF.html", plot=force, full_html=True)

